# Reward, Cost, Termination and Step Information


Following the standard OpenAI Gym API, after each step of the environment `env.step(...)`, the environment will return a tuple containing five items: `(obs, reward, terminated, truncated, info)`. In this page, we discuss the design of reward function `reward`, cost function `info["cost"]`, termination criterion `terminated` in various settings, truncation information `truncated`, and the details of step information `info`.

## Reward Function

For all environments, reward functions consist of generally a dense driving reward and a sparse terminal reward. The dense reward is the longitudinal movement along the reference line or lane toward destination. When the episode is terminated due to, i.e. arriving the destination or driving out of the road, a sparse reward will be added to the dense reward. In practice, the concrete implementations of reward function are slightly different across all environments. 

### MetaDriveEnv
The reward functions for the `MetaDriveEnv` and derived environments like `Multi-agent Environments` and `SafeMetaDriveEnv` are similar as they all using `PGMap`. The implementation is as follows.

In [1]:
from metadrive.envs.metadrive_env import MetaDriveEnv
from metadrive.envs.scenario_env import ScenarioEnv
from metadrive.utils import print_source
print_source(MetaDriveEnv.reward_function)

def reward_function(self, vehicle_id: str):
    """
    Override this func to get a new reward function
    :param vehicle_id: id of BaseVehicle
    :return: reward
    """
    vehicle = self.vehicles[vehicle_id]
    step_info = dict()

    # Reward for moving forward in current lane
    if vehicle.lane in vehicle.navigation.current_ref_lanes:
        current_lane = vehicle.lane
        positive_road = 1
    else:
        current_lane = vehicle.navigation.current_ref_lanes[0]
        current_road = vehicle.navigation.current_road
        positive_road = 1 if not current_road.is_negative_road() else -1
    long_last, _ = current_lane.local_coordinates(vehicle.last_position)
    long_now, lateral_now = current_lane.local_coordinates(vehicle.position)

    # reward for lane keeping, without it vehicle can learn to overtake but fail to keep in lane
    if self.config["use_lateral_reward"]:
        lateral_factor = clip(1 - 2 * abs(lateral_now) / vehicle.navigation.get_current_lane_width(),

This reward function is composed of three parts as follows:

$R = c_{1} R_{driving} + c_{2} R_{speed} + R_{termination}$

- The **driving reward**  $R_{driving} = d_t - d_{t-1}$, wherein the $d_t$ and $d_{t-1}$ denote the longitudinal coordinates of the target vehicle on the current reference lane of two consecutive time steps, providing dense reward to encourage agent to move toward the destination.
- The **speed reward** $R_{speed} = v_t/v_{max}$ incentives agent to drive fast. $v_{t}$ and $v_{max}$ denote the current velocity and the maximum velocity (80 km/h), respectively.
- The **termination reward** $R_{termination}$ contains a set of sparse rewards. At the end of episode, other dense rewards will be disabled and only one sparse reward will be given to the agent at the end of the episode according to its termination state. We implement the `success_reward`, `out_of_road_penalty`, `crash_vehicle_penalty` and `crash_object_penalty` currently. The penalty will be given as negative reward.

We also provide a config call `use_lateral_reward`, which is a multiplier in range [0, 1] indicating whether the ego vehicle is far from the center of current lane. The multiplier will apply to the driving reward.

We summarize the default reward config here:

- `success_reward = 10.0`: one of termination reward.
- `out_of_road_penalty = 5.0`: will use -5.0 as the termination reward.
- `crash_vehicle_penalty = 5.0`: will use -5.0 as the termination reward.
- `crash_object_penalty = 5.0`: will use -5.0 as the termination reward.
- `driving_reward = 1.0`: the $c_{1}$ in reward function.
- `speed_reward = 0.1`: the $c_{2}$ in reward function.
- `use_lateral_reward = False`: disable weighting the driving reward according to centering in the lane.

### ScenarioEnv
The reward function for `ScenarioEnv` is similar to the one of `MetaDriveEnv`, while the calculation of driving reward is slightly different and there are more items. The concrete implementation is:

In [2]:
print_source(ScenarioEnv.reward_function)

def reward_function(self, vehicle_id: str):
    """
    Override this func to get a new reward function
    :param vehicle_id: id of BaseVehicle
    :return: reward
    """
    vehicle = self.vehicles[vehicle_id]
    step_info = dict()

    # Reward for moving forward in current lane
    current_lane = vehicle.lane
    long_last = vehicle.navigation.last_longitude
    long_now = vehicle.navigation.current_longitude
    lateral_now = vehicle.navigation.current_lateral

    # dense driving reward
    reward = 0
    reward += self.config["driving_reward"] * (long_now - long_last)

    # reward for lane keeping, without it vehicle can learn to overtake but fail to keep in lane
    lateral_factor = abs(lateral_now) / self.config["max_lateral_dist"]
    lateral_penalty = -lateral_factor * self.config["lateral_penalty"]
    reward += lateral_penalty

    # heading diff
    ref_line_heading = vehicle.navigation.current_heading_theta_at_long
    heading_diff = wrap_to_pi(abs(vehicle.heading_the

This reward function is composed of six parts as follows:

$R = c_{1} R_{driving} + c_{2} R_{lateral} + c_{3} R_{heading} + c_{4} R_{steering} - c_{5} P_{collision} + R_{termination}$

- The **driving reward** $R_{driving} = d_t - d_{t-1}$, wherein the $d_t$ and $d_{t-1}$ denote the longitudinal movements of the target vehicle on the recorded trajectory of two consecutive time steps, providing dense reward to encourage agent to move toward the destination.
- The **lateral reward** $R_{lateral}$ incentives the agent to driving as close to the reference trajectory as possible. When the distance between the car and the reference trajectory is larger than `max_lateral_dist`, the episode will be terminated due to driving out of road. 
- The **heading reward** $R_{heading}$ asks the agent to have the same heading as the direction of a certain point on the lane. The point is calculated per step by projecting the position of the vehicle to the reference line (recorded trajectory). 
- The **steering reward** $R_{steering}$ is for yielding large steering actions when the speed is high. The higher the speed is, the larger the penalty for the large steering angle will be.
- The **collision penalty** $P_{collision}$ is a constant scalar for penalizing any collision behaviors with certain type of traffic participants.
- The **termination reward** $R_{termination}$ contains a set of sparse rewards. At the end of episode, other dense rewards will be disabled and only one sparse reward will be given to the agent at the end of the episode according to its termination state. We use the `success_reward`, `out_of_road_penalty` for this environment. The penalty will be given as negative reward.

There is a special config `no_negative_reward`. When setting it to `True`, the final reward will be clipped to `reward=max(reward, 0)`. This is helpful for stabilizing the training. We summarize the default reward config of the `ScenarioEnv` here:

- success_reward=5.0,
- out_of_road_penalty=5.0,
- on_lane_line_penalty=1.,
- crash_vehicle_penalty=1.,
- crash_object_penalty=1.0,
- crash_human_penalty=1.0,
- driving_reward=1.0,
- steering_range_penalty=0.5,
- heading_penalty=1.0,
- lateral_penalty=.5,
- max_lateral_dist=4,
- no_negative_reward=True,

It is worth noting that we write some information like the lateral reward and heading reward to the `step_info`. Thus we can keep track of the training status of the agent.

## Cost Function

Similar to the reward function, we also provide default cost function to measure the safety during driving. The cost function will be placed in the returned information dict as `info["cost"]` after `env.step` function.

- `crash_vehicle_cost = 1.0`: yield cost when crashing to other vehicles.
- `crash_object_cost = 1.0`: yield cost when crashing to objects, such as cones and triangles.
- `out_of_road_cost = 1.0`: yield cost when driving out of the road.

The cost function is implemented in the `cost_function` in `MetaDriveEnv <https://github.com/metadriverse/metadrive/blob/main/metadrive/envs/metadrive_env.py>`_.

## Termination Function

MetaDrive will terminate an episode of a vehicle if:

1. the target vehicle arrive its destination,
2. the vehicle drives out of the road,
3. the vehicle crashes to other vehicles,
4. the vehicle crashes to obstacles, or
5. the vehicle crashes to building (e.g. in Multi-agent Tollgate environment).

The above termination criterion is implemented in the `done_function` in `MetaDriveEnv <https://github.com/metadriverse/metadrive/blob/main/metadrive/envs/metadrive_env.py#>`_.

Please note that in the Safe RL environment `SafeMetaDriveEnv <https://github.com/metadriverse/metadrive/blob/main/metadrive/envs/safe_metadrive_env.py>`_, the episode will not be terminated when vehicles crashing into objects or vehicles.
This is because we wish to investigate the safety performance of a vehicle in an extremely dangerous environments.
Terminating episodes too frequently will let the training becomes too hard to complete.

In Multi-agent environment, the above termination criterion is still hold for each vehicle. We call this the termination of an *agent episode*.
We explicitly add two config to adjust the termination processing in MARL: `crash_done = True` and `out_of_road_done = True`.
They denotes whether to terminate the agent episode if crash / out of road happens.

Besides, in Multi-agent environment, the controllable target vehicles consistently respawn in the scene if old target vehicles are terminated.
To limit the length of *environmental episode*, we also introduce a config `horizon = 1000` in MARL environments.
The environmental episode has a **minimal length** of `horizon` steps and the environment will stop spawning new target vehicles if this horizon is exceeded.
If you wish to disable the respawning mechanism in MARL, set the config `allow_respawn = False`. In this case, the environmental episode will terminate if no active vehicles are in the scene.



## Truncation Information

Currently, this is always set to `False` for each vehicle. 

## Step Information

The step information dict `info` contains rich information about current state of the environment and the target vehicle.
We summarize the dict as follows:

.. code-block::

    {
        # Number of vehicles being overtaken by ego vehicle in this episode
        'overtake_vehicle_num': 0,

        # Current velocity in km/h
        'velocity': 0.0,

        # The current normalized steering signal in [-1, 1]
        'steering': -0.06901532411575317,

        # The current normalized acceleration signal in [-1, 1]
        'acceleration': -0.2931942343711853,

        # The normalized action after clipped who is applied to the ego vehicle
        'raw_action': (-0.06901532411575317, -0.2931942343711853),

        # Whether crash to vehicle / object / building
        'crash_vehicle': False,
        'crash_object': False,
        'crash_building': False,
        'crash': False,  # Whether any kind of crash happens

        # Whether going out of the road / arrive destination
        # or exceeding the maximal episode length
        'out_of_road': False,
        'arrive_dest': False,
        'max_step': False,

        # The reward in this time step / the whole episode so far
        'step_reward': 0.0,
        'episode_reward': 0.0,

        # The cost in this time step
        'cost': 0,

        # The length of current episode
        'episode_length': 1
    }

In Safe RL environment `SafeMetaDriveEnv <https://github.com/metadriverse/metadrive/blob/main/metadrive/envs/safe_metadrive_env.py>`_, we additionally record the `info["total_cost"]` to record the sum of all cost within one episode.

The step info is collected from various sources such as the engine, reward function, termination function, traffic manager, agent manager and so on.




